In [56]:
df.iloc[0][0]

/tmp/ipykernel_31/2900375399.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.iloc[0][0]


{'input_ids': tensor([[37523, 11851,    25,  1400,    12,    33,   539, 11959, 45305,   198,
           198, 41222,    25,   198, 14692,    16,   269,    13, 14245, 11856,
          7586,  7543,  1600,   366,    16,    14,    17,   269,    13, 28959,
           515,  7545,  1600,   366,    16,    14,    17, 23053,    13, 16858,
          1600,   366,    16,    14,    17,   269,    13,  5445, 14380,   357,
         43106,   504, 42501,   366,    17,   309, 24145,    13,  9215,   393,
          6145, 34569,  1600,   366,    18,   352,    14,    17,   269,    13,
         13197,  2546, 37624, 11464, 50128,  8973,   198,   198, 43993,   507,
            25,   198, 14692,   818,   257,  4334,   362,    12, 36008, 10746,
          6839,    11,  5022,  7586,  7543,    11, 14380,    11, 28959,   515,
          7545,   290,  9215,   393,  6145, 34569, 33283,   366,  1273,   343,
           625,  7090,  4894,  1566, 11710, 25037,   477,   625,  1353, 33283,
           366,    33,  9437,   290, 1

In [57]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size = 0.1, random_state = 42)

In [58]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [59]:
model = AutoModelForCausalLM.from_pretrained(
                model_name,
                pad_token_id=tokenizer.eos_token_id
            )
            
# Resize token embeddings if needed
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 1024)

In [60]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [61]:
!pip install torchinfo

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [62]:
from torchinfo import summary

summary(model)  # Adjust input_size based on your needs

Layer (type:depth-idx)                             Param #
GPT2LMHeadModel                                    --
├─GPT2Model: 1-1                                   --
│    └─Embedding: 2-1                              51,463,168
│    └─Embedding: 2-2                              1,048,576
│    └─Dropout: 2-3                                --
│    └─ModuleList: 2-4                             --
│    │    └─GPT2Block: 3-1                         12,596,224
│    │    └─GPT2Block: 3-2                         12,596,224
│    │    └─GPT2Block: 3-3                         12,596,224
│    │    └─GPT2Block: 3-4                         12,596,224
│    │    └─GPT2Block: 3-5                         12,596,224
│    │    └─GPT2Block: 3-6                         12,596,224
│    │    └─GPT2Block: 3-7                         12,596,224
│    │    └─GPT2Block: 3-8                         12,596,224
│    │    └─GPT2Block: 3-9                         12,596,224
│    │    └─GPT2Block: 3-10                 

In [63]:
prompt = "Generate a chocolate chip cookie recipe with these ingredients: flour, sugar, eggs, chocolate chips\n\nRecipe Title:"
prompt = tokenizer(prompt,  return_tensors="pt")

params = {
                'max_length': 400,
                'num_return_sequences': 1,
                'temperature': 0.7,
                'top_k': 50,
                'top_p': 0.9,
                'do_sample': True,
                'no_repeat_ngram_size': 2,
                'early_stopping': True
            }

In [64]:
outputs = model.generate(
            **prompt,
            **params
        )

print(outputs)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[ 8645,   378,   257, 11311, 11594, 19751,  8364,   351,   777,  9391,
            25, 10601,    11,  7543,    11,  9653,    11, 11311, 12014,   198,
           198, 37523, 11851,    25, 24777, 17869, 39606, 26694,   198,    11,
          6434,    25, 14235,    11, 26088,    11,   290, 14235,   198,    12,
           352,  6508,   357,    17, 16461,     8,  5576, 29590,  9215,    11,
         41534,   198,   628,   198,    16,  6508,  7543,   198,    13,   764,
           764,   198,   286,  7586,  7543,   764,   628,   262,   976,  2033,
           286, 24178,  9215,   628,    13,   198,   357,    40,   973,   352,
            14,    19,  6508,  7586,  9215,   329,   428,  8364,     8,   628,
          1849,    12,   362, 14180,   477,  4007, 10601,   628,   628,   764,
           357,  5832,   460,   779,   597,  1611,   286, 10601,   345,   588,
            13,   314,   973,  7586,    11,  2330,    11,   393,   772,  2187,
         17135, 10601,  2014,   628,   532,   352,  

In [65]:
generated_text = tokenizer.decode(
    outputs[0], 
    skip_special_tokens=True
)

In [66]:
print(generated_text)

Generate a chocolate chip cookie recipe with these ingredients: flour, sugar, eggs, chocolate chips

Recipe Title: Chocolate Chip Cookie Recipe
, Author: Amy, Christine, and Amy
- 1 cup (2 sticks) unsalted butter, softened



1 cup sugar
. . .
 of brown sugar .

 the same amount of melted butter

.
 (I used 1/4 cup brown butter for this recipe)

 - 2 cups all purpose flour



 . (you can use any kind of flour you like. I used brown, white, or even whole wheat flour.)

 - 1 1⁄2 tsp baking powder
 . (. . you can also use powdered sugar or all-purpose flour)
This recipe can be doubled and you'll get a lot more cookies!

- 3/8 cup granulated sugar

-- 1 tsp vanilla extract
The recipe calls for butter. But if you have a double boiler you could also just use the butter in the oven. However, I did use butter and it was great!
I also used the vanilla and the chocolate. If you don't have vanilla, you will need to use a little more of it. The chocolate is very thin.

I'm not sure if the recipe i

In [67]:
from peft import get_peft_model, LoraConfig, TaskType

In [68]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["c_attn"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [78]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 1024)
        (wpe): Embedding(1024, 1024)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPT2Block(
            (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=3072, nx=1024)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
      

In [69]:
summary(model)

Layer (type:depth-idx)                                            Param #
PeftModelForCausalLM                                              --
├─LoraModel: 1-1                                                  --
│    └─GPT2LMHeadModel: 2-1                                       --
│    │    └─GPT2Model: 3-1                                        355,216,384
│    │    └─Linear: 3-2                                           (51,463,168)
Total params: 406,679,552
Trainable params: 393,216
Non-trainable params: 406,286,336

In [70]:
trainable_params = 393216
total_params = 406679552
percentage_trainable_params = trainable_params * 100 / total_params
print("Percentage of parameters which are trainable:",percentage_trainable_params)

Percentage of parameters which are trainable: 0.09668939538912445


In [71]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [84]:
training_args = TrainingArguments(
    output_dir="./gpt2-lora",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_steps=1,
    save_strategy="epoch",
    fp16=True,  # Set to False if not using GPU with float16 support
)

In [188]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, TrainerCallback, pipeline

class Callbacks(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        if LoRA:
            model.save_pretrained(f'./checkpoint-{state.global_step}/lora/')
        tokenizer.save_pretrained(f'./checkpoint-{state.global_step}/tokenizer/')
        model.config.to_json_file(f'./checkpoint-{state.global_step}/config.json')
    def on_log(self, args, state, control, **kwargs):
        prompt = '你好。<end of turn>'
        chatbot = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)
        model_input = tokenizer.encode(prompt, return_tensors='pt').to('cuda')
        generated_ids = model.generate(input_ids=model_input, max_length=10, pad_token_id=tokenizer.eos_token_id)
        generated_text = tokenizer.decode(generated_ids[:, model_input.shape[-1]:][0], skip_special_tokens=True)
        print(f"Response to '{prompt}':", generated_text)


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, TrainerCallback, pipeline

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Current Device: ', device)

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model = model.to(device)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
model.resize_token_embeddings(len(tokenizer))
model = torch.compile(model)

In [ ]:
config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    # tune the embedding layer and prediction head
    modules_to_save=["wte", "lm_head"]
)
model = get_peft_model(model, config)

In [ ]:
training_args = TrainingArguments(
    output_dir="/home/baily/eureka/gpt-2-lora",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    logging_steps=200,
    save_steps=2000,
    save_total_limit=10,
    warmup_steps=5000,
    gradient_accumulation_steps=2,
    learning_rate=3e-5,
    fp16=True,
    )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_df,
    eval_dataset=test_df,
    callbacks=[Callbacks(),]
)
trainer.train()

In [34]:
!pip install git+https://github.com/huggingface/trl.git
!pip install bitsandbytes>=0.44.0
!pip install trl==0.12.2

  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-a70rmg0o
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-a70rmg0o
  Resolved https://github.com/huggingface/trl.git to commit cc044e35b285be7dc062764b3364e1e684db4c7c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.18.0.dev0-py3-none-any.whl size=350386 sha256=76bd4290bffe9ebeb2d5d27e6c34bdd0665b5885e2cf8724d859c85b8bb6994f
  Stored in directory: /tmp/pip-ephem-wheel-cache-uitck95r/wheels/e7/e5/ec/8cce76372e10e954b47f3893a36ff7c0786d9dbc846efa8860
Successfully built trl
  Attempting uninstall: trl
    Found existing installation: trl 0.12.0
    Uninstalling trl-0.12.0:
      Successfully uninstalled trl-0.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [50]:
from trl import SFTTrainer
import torch
import pandas as pd
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer

In [12]:
num_samples = 1000

df = pd.read_csv("/kaggle/input/recipenlg/RecipeNLG_dataset.csv", nrows = num_samples)
df = df.drop(columns=['Unnamed: 0', 'link', 'source'])

In [13]:
formatted_texts = []

for index, row in df.iterrows():
    title = row['title']
    ingredients = row['ingredients']
    directions = row['directions']
    ner = row['NER']

    text = (
        f"Recipe Title: {title}\n\n"
        f"Ingredients:\n{ingredients}\n\n"
        f"Instructions:\n{directions}\n\n"
        f"Food Entities: {ner}"
    )

    formatted_texts.append(text)

df['text'] = formatted_texts

In [14]:
df.head()

,title,ingredients,directions,NER,text
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...",Recipe Title: No-Bake Nut Cookies\n\nIngredien...
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom...",Recipe Title: Jewell Ball'S Chicken\n\nIngredi...
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar...","Recipe Title: Creamy Corn\n\nIngredients:\n[""2..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo...",Recipe Title: Chicken Funny\n\nIngredients:\n[...
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu...",Recipe Title: Reeses Cups(Candy) \n\nIngredie...


In [15]:
model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [45]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map={"": 0}, torch_dtype=torch.float16)

model = prepare_model_for_kbit_training(model)

In [27]:
peft_config = LoraConfig(r=4, lora_alpha=16, lora_dropout=0.1, bias='none', task_type='CAUSAL_LM')
model = get_peft_model(model, peft_config)

In [54]:
training_args = TrainingArguments(
        output_dir="/home/baily/eureka/gpt-2-lora",
        per_device_train_batch_size=4,
        optim="adamw_torch",
        logging_steps=100,
        learning_rate=2e-4,
        fp16=True,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
        num_train_epochs=1,
        save_strategy="epoch",
        push_to_hub=False,
)

In [2]:
%pip install peft==0.4.0

Note: you may need to restart the kernel to use updated packages.


In [3]:
mkdir cache

In [33]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

#model_name = "bigscience/bloomz-560m"
model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)

In [34]:
import pandas as pd

num_samples = 100000

df = pd.read_csv("/kaggle/input/recipenlg/RecipeNLG_dataset.csv", nrows = num_samples)
df = df.drop(columns=['Unnamed: 0', 'link', 'source'])

In [35]:
formatted_texts = []

for index, row in df.iterrows():
    title = row['title']
    ingredients = row['ingredients']
    directions = row['directions']
    ner = row['NER']

    text = (
        f"Recipe Title: {title}\n\n"
        f"Ingredients:\n{ingredients}\n\n"
        f"Instructions:\n{directions}\n\n"
        f"Food Entities: {ner}"
    )

    formatted_texts.append(text)

df['text'] = formatted_texts

In [36]:
df.head()

,title,ingredients,directions,NER,text
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...",Recipe Title: No-Bake Nut Cookies\n\nIngredien...
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom...",Recipe Title: Jewell Ball'S Chicken\n\nIngredi...
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar...","Recipe Title: Creamy Corn\n\nIngredients:\n[""2..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo...",Recipe Title: Chicken Funny\n\nIngredients:\n[...
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu...",Recipe Title: Reeses Cups(Candy) \n\nIngredie...


In [37]:
ids = []
a_masks = []

max_length = 256

for index, row in df.iterrows():
    tokenized_text = tokenizer(
                    row['text'],
                    truncation=True,
                    padding="max_length",
                    max_length=max_length,
                    return_tensors="pt"
                    )

    ids.append(tokenized_text['input_ids'])
    a_masks.append(tokenized_text['attention_mask'])

In [38]:
ids[0].shape

torch.Size([1, 256])

In [39]:
a_masks[0].shape

torch.Size([1, 256])

In [40]:
import torch

input_id = torch.squeeze(ids[0], dim=0)
print(input_id.shape)

torch.Size([256])


In [41]:
final_ids = []
final_a_masks = []

for i in ids:
    input_ids = torch.squeeze(i, dim=0)
    final_ids.append(input_ids.tolist())

for i in a_masks:
    a_mask = torch.squeeze(i, dim=0)
    final_a_masks.append(a_mask.tolist())

In [42]:
df['input_ids'] = final_ids
df['attention_masks'] = final_a_masks

In [43]:
df.head()

,title,ingredients,directions,NER,text,input_ids,attention_masks
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...",Recipe Title: No-Bake Nut Cookies\n\nIngredien...,"[37523, 11851, 25, 1400, 12, 33, 539, 11959, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom...",Recipe Title: Jewell Ball'S Chicken\n\nIngredi...,"[37523, 11851, 25, 3370, 695, 6932, 6, 50, 164...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar...","Recipe Title: Creamy Corn\n\nIngredients:\n[""2...","[37523, 11851, 25, 19979, 88, 11424, 198, 198,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo...",Recipe Title: Chicken Funny\n\nIngredients:\n[...,"[37523, 11851, 25, 16405, 40473, 198, 198, 412...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu...",Recipe Title: Reeses Cups(Candy) \n\nIngredie...,"[37523, 11851, 25, 797, 274, 274, 41611, 7, 34...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [44]:
from datasets import Dataset

df = Dataset.from_pandas(df)

print(df)
print(type(df))  # Should output: <class 'datasets.arrow_dataset.Dataset'>

Dataset({
    features: ['title', 'ingredients', 'directions', 'NER', 'text', 'input_ids', 'attention_masks'],
    num_rows: 100000
})
<class 'datasets.arrow_dataset.Dataset'>


In [45]:
import peft
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=2,
    lora_alpha=8,
    target_modules=["c_attn"],
    lora_dropout=0.05, 
    bias="none",
    task_type="CAUSAL_LM"
)

In [46]:
peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.print_trainable_parameters())

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora.py:299: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 196,608 || all params: 355,019,776 || trainable%: 0.05537945018589612
None


In [47]:
import transformers
from transformers import TrainingArguments, Trainer
import os

output_directory = os.path.join("../cache/working", "peft_lab_outputs")
training_args = TrainingArguments(
    report_to="none",
    output_dir=output_directory,
    auto_find_batch_size=True,
    learning_rate= 3e-2, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=100,
    #use_cpu=True
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=df,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,3.305700
200,3.176600
300,3.316000
400,3.203800
500,3.068200
600,3.302200
700,3.221000
800,3.088900
900,3.282200
1000,3.684700


TrainOutput(global_step=6250, training_loss=3.6223760107421876, metrics={'train_runtime': 6983.7067, 'train_samples_per_second': 14.319, 'train_steps_per_second': 0.895, 'total_flos': 4.64652337152e+16, 'train_loss': 3.6223760107421876, 'epoch': 1.0})